In [8]:
# Import packages
import osgeo
from os.path import join as pjoin
import pandas as pd
import numpy as np
import xarray as xr
import xrspatial as xrs
import rioxarray

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns

import pystac
from pystac_client import Client
import pystac.item_collection as pyic
import planetary_computer
import requests
import stackstac

import rich.table
import dask.diagnostics

scratch_dir = './Project/scratch' 

from pystac_client import Client
import planetary_computer as pc

In [59]:
# Define your area of interest
aoi_BMV = { #aoi Everest and Barun/Makalu national park polygon
  "type": "Polygon",
  "coordinates": [
    [
        [86.736074, 28.136264],
        [87.181495, 27.970180],
        [87.276902, 27.567789],
        [87.276902, 28.136264],
        [86.736074, 28.136264]
    ]
  ]
}

In [62]:
daterange = [
    #{"interval": ["1983-09-01T00:00:00Z", "1983-11-01T00:00:00Z"]}, geen data
    #{"interval": ["1986-09-01T00:00:00Z", "1986-11-01T00:00:00Z"]}, geen data
    #{"interval": ["1989-08-01T00:00:00Z", "1989-11-01T00:00:00Z"]}, geen data
    {"interval": ["1992-09-01T00:00:00Z", "1992-11-01T00:00:00Z"]},
    {"interval": ["1995-09-01T00:00:00Z", "1995-11-01T00:00:00Z"]},
    {"interval": ["1998-09-01T00:00:00Z", "1998-11-01T00:00:00Z"]},
    {"interval": ["2001-09-01T00:00:00Z", "2001-11-01T00:00:00Z"]},
    {"interval": ["2004-09-01T00:00:00Z", "2004-11-01T00:00:00Z"]},
    {"interval": ["2007-09-01T00:00:00Z", "2007-11-01T00:00:00Z"]},
    {"interval": ["2010-09-01T00:00:00Z", "2010-11-01T00:00:00Z"]},
    {"interval": ["2013-09-01T00:00:00Z", "2013-11-01T00:00:00Z"]},
    {"interval": ["2016-09-01T00:00:00Z", "2016-11-01T00:00:00Z"]},
    {"interval": ["2019-09-01T00:00:00Z", "2019-11-01T00:00:00Z"]},
    {"interval": ["2022-09-01T00:00:00Z", "2022-11-01T00:00:00Z"]}]

def importdata(aoi, daterange):
    catalog = Client.open(
        "https://planetarycomputer.microsoft.com/api/stac/v1"
        )
    # Define your search with CQL2 syntax
    search = catalog.search(filter_lang="cql2-json", filter={
        "op": "and",
        "args": [{"op": "s_intersects", "args": [{"property": "geometry"}, aoi]},
                {"op": "anyinteracts", "args": [{"property": "datetime"}, daterange[0]]},
                {"op": "=", "args": [{"property": "collection"}, "landsat-c2-l2"]},
                {"op": "<=", "args": [{"property": "eo:cloud_cover"}, 20]}
                ]
        }
    )


    first_item = next(search.get_items())
    pc.sign_item(first_item).assets

    charts = search.get_all_items()
    print(daterange[0],len(charts))
               
        #########################
    for t in daterange[1:]:
         # Search against the Planetary Computer STAC API
        catalog = Client.open(
            "https://planetarycomputer.microsoft.com/api/stac/v1")
        # Define your search with CQL2 syntax
        search = catalog.search(filter_lang="cql2-json", filter={
            "op": "and",
                "args": [
                    {"op": "s_intersects", "args": [{"property": "geometry"}, aoi]},
                    {"op": "anyinteracts", "args": [{"property": "datetime"}, t]},
                    {"op": "=", "args": [{"property": "collection"}, "landsat-c2-l2"]},
                    # data cant process more than k-amount of bands so cloud coverage can be set to almost 0
                    {"op": "<=", "args": [{"property": "eo:cloud_cover"}, 20]} 
                ]
            }  
        )

        first_item = next(search.get_items())
        pc.sign_item(first_item).assets

        items = search.get_all_items()
        print(t,'items: ',len(items))
        charts = charts+items
        

    print('Length total item set:',len(charts)) 
    return charts    

charts = importdata(aoi_BMV, daterange)

{'interval': ['1992-09-01T00:00:00Z', '1992-11-01T00:00:00Z']} 3
{'interval': ['1995-09-01T00:00:00Z', '1995-11-01T00:00:00Z']} items:  5
{'interval': ['1998-09-01T00:00:00Z', '1998-11-01T00:00:00Z']} items:  3
{'interval': ['2001-09-01T00:00:00Z', '2001-11-01T00:00:00Z']} items:  2
{'interval': ['2004-09-01T00:00:00Z', '2004-11-01T00:00:00Z']} items:  6
{'interval': ['2007-09-01T00:00:00Z', '2007-11-01T00:00:00Z']} items:  4
{'interval': ['2010-09-01T00:00:00Z', '2010-11-01T00:00:00Z']} items:  4
{'interval': ['2013-09-01T00:00:00Z', '2013-11-01T00:00:00Z']} items:  5
{'interval': ['2016-09-01T00:00:00Z', '2016-11-01T00:00:00Z']} items:  5
{'interval': ['2019-09-01T00:00:00Z', '2019-11-01T00:00:00Z']} items:  4
{'interval': ['2022-09-01T00:00:00Z', '2022-11-01T00:00:00Z']} items:  6
Length total item set: 47


In [61]:
ds = stackstac.stack(planetary_computer.sign(charts), epsg=6207)


In [53]:
xmin, xmax, ymin, ymax = 85.527584483, 88.400149020, 27.250970920, 29.916814171 # Set to a small area to limit computation time.
ds2 = ds.loc[:,:, ymax:ymin,xmin:xmax]
ds2

<xarray.DataArray 'stackstac-4faf8c5810c1886f7e01eea37908baf5' (time: 49,
                                                                band: 22,
                                                                y: 9763, x: 9270)>
dask.array<getitem, shape=(49, 22, 9763, 9270), dtype=float64, chunksize=(1, 1, 1024, 1024), chunktype=numpy.ndarray>
Coordinates: (12/31)
  * time                         (time) datetime64[ns] 1992-08-13T04:04:46.74...
    id                           (time) <U31 'LT05_L2SP_140041_19920813_02_T1...
  * band                         (band) <U13 'qa' 'red' ... 'qa_aerosol'
  * x                            (x) float64 85.53 85.53 85.53 ... 88.4 88.4
  * y                            (y) float64 29.92 29.92 29.92 ... 27.25 27.25
    landsat:wrs_row              (time) <U3 '041' '041' '040' ... '040' '041'
    ...                           ...
    raster:bands                 (band) object {'unit': 'kelvin', 'scale': 0....
    classification:bitfields     (band) object None ... [{'name': 'fill', 'le...
    common_name                  (band) object None None None ... 'coastal' None
    center_wavelength            (band) object None None None ... 10.9 0.44 None
    full_width_half_max          (band) object None None None ... 0.59 0.02 None
    epsg                         int32 6207
Attributes:
    spec:           RasterSpec(epsg=6207, bounds=(85.5302511826651, 26.365303...
    crs:            epsg:6207
    transform:      | 0.00, 0.00, 85.53|\n| 0.00,-0.00, 29.92|\n| 0.00, 0.00,...
    resolution_xy:  (0.00030960840672378254, 0.0002730345003136847)

In [5]:
import rich.table

table = rich.table.Table("Asset Key", "Description")
for asset_key, asset in charts[0].assets.items():
    table.add_row(asset_key, asset.title)

table

┏━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Asset Key        ┃ Description                                                      ┃
┡━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ qa               │ Surface Temperature Quality Assessment Band                      │
│ ang              │ Angle Coefficients File                                          │
│ red              │ Red Band                                                         │
│ blue             │ Blue Band                                                        │
│ drad             │ Downwelled Radiance Band                                         │
│ emis             │ Emissivity Band                                                  │
│ emsd             │ Emissivity Standard Deviation Band                               │
│ lwir             │ Surface Temperature Band                                         │
│ trad             │ Thermal Radiance Band                                            │
│ urad             │ Upwelled Radiance Band                                           │
│ atran            │ Atmospheric Transmittance Band                                   │
│ cdist            │ Cloud Distance Band                                              │
│ green            │ Green Band                                                       │
│ nir08            │ Near Infrared Band 0.8                                           │
│ swir16           │ Short-wave Infrared Band 1.6                                     │
│ swir22           │ Short-wave Infrared Band 2.2                                     │
│ mtl.txt          │ Product Metadata File (txt)                                      │
│ mtl.xml          │ Product Metadata File (xml)                                      │
│ cloud_qa         │ Cloud Quality Assessment Band                                    │
│ mtl.json         │ Product Metadata File (json)                                     │
│ qa_pixel         │ Pixel Quality Assessment Band                                    │
│ qa_radsat        │ Radiometric Saturation and Dropped Pixel Quality Assessment Band │
│ atmos_opacity    │ Atmospheric Opacity Band                                         │
│ tilejson         │ TileJSON with default rendering                                  │
│ rendered_preview │ Rendered preview                                                 │
└──────────────────┴──────────────────────────────────────────────────────────────────┘

In [54]:
TM4 = ds2.sel(band = 'blue')
TM5 = ds2.sel(band = 'drad')
ding = TM4/TM5
resampled = ding.resample(time="YS").max("time", keep_attrs=True)


c:\Users\Jurrian\anaconda3\envs\ssml\lib\site-packages\dask\array\core.py:4828: PerformanceWarning: Increasing number of chunks by factor of 10
  result = blockwise(
c:\Users\Jurrian\anaconda3\envs\ssml\lib\site-packages\dask\array\core.py:4828: PerformanceWarning: Increasing number of chunks by factor of 10
  result = blockwise(


In [55]:
resampled

<xarray.DataArray 'stackstac-4faf8c5810c1886f7e01eea37908baf5' (time: 31,
                                                                y: 9763, x: 9270)>
dask.array<where, shape=(31, 9763, 9270), dtype=float64, chunksize=(3, 1024, 1024), chunktype=numpy.ndarray>
Coordinates: (12/15)
  * x                          (x) float64 85.53 85.53 85.53 ... 88.4 88.4 88.4
  * y                          (y) float64 29.92 29.92 29.92 ... 27.25 27.25
  * time                       (time) datetime64[ns] 1992-01-01 ... 2022-01-01
    landsat:correction         <U4 'L2SP'
    landsat:wrs_path           <U3 '140'
    landsat:wrs_type           <U1 '2'
    ...                         ...
    proj:epsg                  int32 32645
    classification:bitfields   object None
    common_name                object None
    center_wavelength          object None
    full_width_half_max        object None
    epsg                       int32 6207

In [56]:
from IPython.display import Image

Image(url=charts[0].assets["rendered_preview"].href, width=500)

In [ ]:
with dask.diagnostics.ProgressBar():
    ts = ds2.compute()

ts.rio.to_raster(pjoin(scratch_dir, 'ts.tif'), compress='LZW')